# Imports

In [1]:
!pip install -U tensorflow==2.6.4
!pip install cloud-tpu-client

     |████████████████████████████████| 464.2 MB 9.0 kB/s  eta 0:00:01     |███████████████████████▏        | 336.3 MB 67.2 MB/s eta 0:00:02
     |████████████████████████████████| 4.8 MB 40.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 28.8 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 60.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 59.2 MB/s eta 0:00:01
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30692 sha256=1625101a82bab912acfd9f75f7868fad8516ceac35d744c788979e06cbb7419d
  Stored in directory: /root/.cache/pip/wheels/98/91/04/971b4c587cf47ae952b108949b46926f426c02832d120a082a
Successfully built clang
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:


In [2]:
!pip install -qq /kaggle/input/keras-cv-attention-models/keras_cv_attention_models-1.3.9-py3-none-any.whl

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pydicom
import os
import cv2
import tensorflow as tf
from cloud_tpu_client import Client
from tensorflow import keras
import tensorflow_addons as tfa
from keras_cv_attention_models import convnext
from tensorflow.keras import backend as K
from google.cloud import storage

2023-02-07 20:09:03.862631: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-02-07 20:09:03.863363: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Ad

In [4]:
tf.version.VERSION

'2.6.4'

In [5]:
from cloud_tpu_client import Client
try:
    Client().configure_tpu_version(tf.__version__, restart_type='ifNeeded')
except ValueError as e:
    print(e)

# Define GCS Storage Area

In [6]:
client_area = 'kagglersna01'
storage_client = storage.Client(project=client_area)

# Delete All

In [7]:
def delete_file(filepath):
    print('deleting ' + filepath + ' from local')
    os.remove('/kaggle/working/' + filepath)
    
def clear_all_local():
    for k in os.listdir('/kaggle/working/'):
        if k == '.virtual_documents':
            continue
        delete_file(k)

In [8]:
import shutil

In [9]:
def clear_all():
    shutil.rmtree('/kaggle/working/checkpoints')
    clear_all_local()
#clear_all()

# Init TPU

In [10]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470


2023-02-07 20:09:29.714536: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-02-07 20:09:29.714606: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-07 20:09:29.714646: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a97d3caee48c): /proc/driver/nvidia/version does not exist
2023-02-07 20:09:29.717084: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-07 20:09:29.734140: I tensorflow/core/distributed_runtime/rpc/grpc_ch

REPLICAS:  8


# Setup Params

In [11]:
imsize = [512, 512, 3] 
epoch = 50
batchsize = 16 * strategy.num_replicas_in_sync

trim = 43764 
validim = 10942 
step_per_epoch = (trim // batchsize)
steps_per_val_epoch = (validim // batchsize) + 1

wd = 0.01
lr = 5e-6 * strategy.num_replicas_in_sync
xyc = 0.5

# Parsing Function

In [12]:
def parse_example(tfrecord):
    feature_desc = {
        'image': tf.io.FixedLenFeature([], tf.string, default_value=""),
        'label': tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        'age': tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        'impant': tf.io.FixedLenFeature([], tf.int64, default_value=-1),
        'laterality': tf.io.FixedLenFeature([], tf.string, default_value=""),
        'view': tf.io.FixedLenFeature([], tf.string, default_value=""),
        'diff_neg': tf.io.FixedLenFeature([], tf.int64, default_value=-1)
    }
    
    example = tf.io.parse_single_example(tfrecord, feature_desc)
    image = tf.io.parse_tensor(example["image"], out_type=tf.uint8)
    image = tf.reshape(image, shape=imsize)
    return image, tf.cast(example["label"], tf.float32)

# Data Augmentations

> **INPUT** - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3] 

> **OUTPUT** - image with CT squares of side size SZ*DIM removed

In [13]:
def dropout(image, DIM=512, PROBABILITY=0.6, CT=5, SZ=0.10):
    
    # DO DROPOUT WITH PROBABILITY DEFINED ABOVE
    P = tf.cast( tf.random.uniform([], 0, 1)<PROBABILITY, tf.int32)
    if (P==0) | (CT==0) | (SZ==0): 
        return image
    
    for k in range(CT):
        
        # CHOOSE RANDOM LOCATION
        x = tf.cast(tf.random.uniform([], 0, DIM),tf.int32)
        y = tf.cast(tf.random.uniform([], 0, DIM),tf.int32)
        
        # COMPUTE SQUARE 
        WIDTH = tf.cast(SZ*DIM, tf.int32) * P
        ya = tf.math.maximum(0, y-WIDTH//2)
        yb = tf.math.minimum(DIM, y+WIDTH//2)
        xa = tf.math.maximum(0, x-WIDTH//2)
        xb = tf.math.minimum(DIM, x+WIDTH//2)
        
        # DROPOUT IMAGE
        one = image[ya:yb, 0:xa, :]
        two = tf.zeros([yb-ya, xb-xa, 3], dtype=image.dtype) 
        three = image[ya:yb, xb:DIM, :]
        middle = tf.concat([one, two, three], axis=1)
        image = tf.concat([image[0:ya,:,:], middle, image[yb:DIM, :, :]], axis=0)
        image = tf.reshape(image, [DIM, DIM, 3])

    return image

In [14]:
data_augm_lay = keras.Sequential(
    [
        keras.layers.RandomZoom(0.3)
    ]
)

In [15]:
def augment_image(image, y):
    
    image = tf.image.random_brightness(image, 0.10)
    image = tf.image.random_contrast(image, 0.90, 1.40)
    image = tf.image.random_saturation(image, 0.50, 2.00)
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    image = data_augm_lay(image)
    image = dropout(image)
    
    return image, y

In [16]:
sample_per = 1 - (2.0770496298327394 / 10)

In [17]:
def undersample_majority(x, y):
    return y == 1 or tf.random.uniform([]) > sample_per

# Get Dataset From GCS Files

In [18]:
def record_dataset(filepaths, shuffle_buffer_size=5000, batch_size=32, training=True, ordered=False):
    
    ignore_order = tf.data.Options()
    
    if not ordered:
        ignore_order.experimental_deterministic = False
    
    dataset = tf.data.TFRecordDataset(filepaths, num_parallel_reads=tf.data.AUTOTUNE)
    
    # dataset = dataset.cache()
    
    dataset = dataset.map(parse_example, num_parallel_calls=tf.data.AUTOTUNE)
    
    if training:
        dataset = dataset.filter(undersample_majority)
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.with_options(ignore_order)
        dataset = dataset.shuffle(shuffle_buffer_size)
        dataset = dataset.repeat()
        
    dataset = dataset.batch(batchsize)
    
    dataset = dataset.prefetch(1)
    return dataset

In [19]:
trainpaths = ['gs://' + 'train_batches' + '/' + "{}_batch_{:0>3}.tfrecord".format('train', index) for index in range(0, 200)]
validpaths = ['gs://' + 'train_batches' + '/' + "{}_batch_{:0>3}.tfrecord".format('valid', index) for index in range(0, 100)]

train_set = record_dataset(trainpaths, batchsize)
valid_set = record_dataset(validpaths, batchsize, training=False)

# Metric/s

In [20]:
class pFBeta(tf.keras.metrics.Metric):
    """Compute overall probabilistic F-beta score."""
    def __init__(self, beta=1, epsilon=1e-5, name='pF1', **kwargs):
        super().__init__(name=name, **kwargs)
        self.beta = beta
        self.epsilon = epsilon
        self.pos = self.add_weight(name='pos', initializer='zeros')
        self.ctp = self.add_weight(name='ctp', initializer='zeros')
        self.cfp = self.add_weight(name='cfp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 0, 1)
        pos = tf.reduce_sum(y_true)
        ctp = tf.reduce_sum(y_pred[y_true==1])
        cfp = tf.reduce_sum(y_pred[y_true==0])
        self.pos.assign_add(pos)
        self.ctp.assign_add(ctp)
        self.cfp.assign_add(cfp)

    def result(self):
        beta_squared = self.beta * self.beta
        c_precision = self.ctp / (self.ctp + self.cfp + self.epsilon)
        c_recall = self.ctp / (self.pos + self.epsilon)
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall)
        return tf.cond(c_precision > 0 and c_recall > 0, lambda: result, lambda: 0.0)

In [21]:
class pFBeta2(tf.keras.metrics.Metric):
    def __init__(self, beta=1, name='pF1', **kwargs):
        super().__init__(name=name, **kwargs)
        self.beta = beta
        self.epsilon = 1e-10
        self.pos = self.add_weight(name='pos', initializer='zeros')
        self.ctp = self.add_weight(name='ctp', initializer='zeros')
        self.cfp = self.add_weight(name='cfp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, 0, 1)
        pos = tf.cast(tf.reduce_sum(y_true), tf.float32)
        ctp = tf.cast(tf.reduce_sum(y_pred[y_true == 1]), tf.float32)
        cfp = tf.cast(tf.reduce_sum(y_pred[y_true == 0]), tf.float32)
        self.pos.assign_add(pos)
        self.ctp.assign_add(ctp)
        self.cfp.assign_add(cfp)

    def result(self):
        beta2 = self.beta * self.beta
        prec = self.ctp / (self.ctp + self.cfp + self.epsilon)
        reca = self.ctp / (self.pos + self.epsilon)
        return (1 + beta2) * prec * reca / (beta2 * prec + reca)

    def reset_state(self):
        self.pos.assign(0.)
        self.ctp.assign(0.)
        self.cfp.assign(0.)

> Find Best Threshold

In [22]:
def pfbeta_tf(labels, preds, beta=1):
    eps = 1e-5
    preds = tf.clip_by_value(preds, 0, 1)
    y_true_count = tf.reduce_sum(labels)
    ctp = tf.reduce_sum(preds[labels==1])
    cfp = tf.reduce_sum(preds[labels==0])
    beta_squared = beta * beta
    c_precision = ctp / (ctp + cfp + eps)
    c_recall = ctp / (y_true_count + eps)
    if (c_precision > 0 and c_recall > 0):
        result = (1 + beta_squared) * (c_precision * c_recall) / (beta_squared * c_precision + c_recall + eps)
        return result
    else:
        return tf.constant(0.0, dtype=tf.float32)

# finds best pf1 using thresholds
def pfbeta_thr(labels, preds):
    thrs = tf.range(0, 1, 0.05)
    best_score = tf.constant(0, dtype=tf.float32)
    for thr in thrs:
        score = pfbeta_tf(labels, tf.cast(preds>thr, tf.float32))
        best_score = tf.cond(score > best_score, lambda: score, lambda: best_score)
    return best_score

pfbeta_thr.__name__='pF1_thr'

# Build Model

In [25]:
def norm(image):
    image = tf.cast(image, tf.float32)
    image = tf.keras.applications.imagenet_utils.preprocess_input(image, mode='tf')
    return image

In [26]:
print(os.listdir('./model/'))

['resnet101-V1.h5', 'resnet101-V5(overfit).h5', 'resnet101-V4.h5', 'resnet101-V3.h5', 'resnet101-V2.h5']


In [27]:
from_saved = False
model_name = 'resnet101-V4.h5'

In [28]:
def get_utility():
    metrics = [
        pFBeta2(name='pF1'),
        pfbeta_thr,
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.BinaryAccuracy()
    ]
    
    lr_obj = tf.keras.optimizers.schedules.CosineDecay(
        lr, 50, alpha=0.0, name='cos_lr'
    )
    
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
    optimizer = keras.optimizers.AdamW(learning_rate=lr_obj, weight_decay=wd)
    
    return metrics, loss, optimizer

In [29]:
if from_saved:    
    with strategy.scope():
        load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
        model = tf.keras.models.load_model(
            './model/' + model_name, 
            options=load_locally, 
            custom_objects={
                "ConvUnit": ConvUnit, 
                "IdentityUnit": IdentityUnit, 
                'pFBeta': pFBeta2,
                'pF1_thr': pfbeta_thr
            }
        )
        
        metrics, loss, optimizer = get_utility()
        
else:
    with strategy.scope():
        model = keras.models.Sequential()
        model.add(keras.layers.InputLayer(input_shape=imsize))
        model.add(keras.layers.Lambda(lambda x: norm(x)))
        model.add(convnext.ConvNeXtV2Tiny(
            input_shape=imsize,
            pretrained='imagenet21k-ft1k',
            num_classes=0))
        model.add(keras.layers.GlobalAvgPool2D())
        model.add(keras.layers.Dense(512, activation="relu"))
        model.add(keras.layers.Dropout(0.30))
        model.add(keras.layers.Dense(1, activation='sigmoid'))

        metrics, loss, optimizer = get_utility()

115154944/115145456 [==============================] - 11s 0us/step
>>>> Load pretrained from: /root/.keras/models/convnext_v2_tiny_384_imagenet21k-ft1k.h5


In [ ]:
lr_obj = tf.keras.optimizers.schedules.CosineDecay(
    lr, 50, alpha=0.0, name='cos_lr'
)

In [31]:
LR_MAX = 5e-6 * strategy.num_replicas_in_sync
CYCLE = 0.5

def lr_fn(step, epochs=epoch, warmup=5, lr_max=LR_MAX, cycle=CYCLE):
    if step < warmup:
        lr = lr_max * 0.10 ** (warmup - step)
    else:
        progress = float(step - warmup) / float(max(1, epochs - warmup))
        lr = max(0.0, 0.5 * (1.0 + np.cos(2.0 * np.pi * float(cycle) * progress))) * lr_max
    return lr

In [32]:
lr_schedule = [lr_fn(step) for step in range(epoch)]
lr_callback = tf.keras.callbacks.LearningRateScheduler(
    lambda step: lr_schedule[step], verbose=0)

In [33]:
model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics
)

In [34]:
tf.keras.backend.clear_session()

tf.config.optimizer.set_jit(True)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, 512, 512, 3)       0         
_________________________________________________________________
convnext_v2_tiny (Functional (None, 16, 16, 768)       27864960  
_________________________________________________________________
global_average_pooling2d (Gl (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               393728    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 28,259,201
Trainable params: 28,259,201
Non-trainable params: 0
__________________________________________

# Train

In [35]:
early_stopping_cb = keras.callbacks.EarlyStopping(
    patience=10, 
    restore_best_weights=True,
    start_from_epoch=5,
    mode='max',
    monitor='val_pF1'
)
history = model.fit(
    train_set, 
    steps_per_epoch=step_per_epoch,
    epochs=epoch, 
    validation_data=valid_set,
    validation_steps=steps_per_val_epoch,
    class_weight = {
        0: 1.0,
        1: 10.0
    },
    callbacks=[early_stopping_cb, lr_callback, wd_callback]
)

Epoch 1/50
Learning rate: 4.00e-05
Weight decay: 4.00e-07
341/341 [==============================] - 248s 457ms/step - loss: 1.2572 - pF1: 0.1687 - pF1_thr: 0.3063 - precision: 0.1154 - recall: 0.5757 - binary_accuracy: 0.5550 - val_loss: 0.5368 - val_pF1: 0.0567 - val_pF1_thr: 0.1415 - val_precision: 0.0499 - val_recall: 0.5060 - val_binary_accuracy: 0.7697
Epoch 2/50
Learning rate: 4.00e-05
Weight decay: 4.00e-07
341/341 [==============================] - 146s 427ms/step - loss: 1.1368 - pF1: 0.2023 - pF1_thr: 0.4066 - precision: 0.1632 - recall: 0.6387 - binary_accuracy: 0.6608 - val_loss: 0.5137 - val_pF1: 0.0658 - val_pF1_thr: 0.1468 - val_precision: 0.0540 - val_recall: 0.5984 - val_binary_accuracy: 0.7523
Epoch 3/50
Learning rate: 3.98e-05
Weight decay: 3.98e-07
341/341 [==============================] - 129s 379ms/step - loss: 1.0077 - pF1: 0.2371 - pF1_thr: 0.4702 - precision: 0.2021 - recall: 0.6948 - binary_accuracy: 0.7187 - val_loss: 0.4622 - val_pF1: 0.0765 - val_pF1_thr:

In [36]:
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
model.save('./model/resnet101-V1.h5', options=save_locally)